<a href="https://colab.research.google.com/github/jhaabh/verus/blob/main/notebooks/Prepare_Atoms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Making an LLM Think Like Marcus Aurelius

## What We're Trying to Do

I'm aiming to train an LLM to think and behave like Marcus Aurelius. The plan is to use a decently smart base model (like mistral 7b v3) and fine-tune it on synthetic data we'll generate from his diary, "Meditations".

## Why This Approach?

"Meditations" is about 40k tokens long. I've tried training on raw chunks of the text, and while it does boost the model's knowledge about Marcus Aurelius, it's not enough. We need to dig deeper - focusing on facts, behavior, ethics, and thinking patterns. Even for simple facts, we need to hammer them in by training on various tasks based on single or combined facts.

Just breaking the text into paragraphs or smaller chunks doesn't cut it. You often end up with bits that aren't self-contained because they reference earlier parts.

So, I'm thinking of rewriting the whole thing into a series of bullet points - each one self-contained and independent as much as possible. This way, we can work with individual points or combine them to generate more synthetic data and tasks (like ethics scenarios, belief systems, factual Q&As, case studies, dialogues, and so on).

## The Process

1. **Prepping the Text**
   - Grab the raw XHTML of "Meditations".
   - Pull out the main text, keeping the structure (books, paragraphs).
   - Chop long paragraphs into smaller bits (max 1500 characters).
   - Mix in the endnotes with the main text for context.

2. **Extracting "Atoms"**
   - For each chunk:
     a. Spot the subjects, topics, entities, and any specialized terms.
     b. Define any fancy terminology.
     c. Create "atoms" - self-contained bits of info on a single topic.
   - Use a beefy LLM (like GPT-4) for this, with a carefully crafted prompt.
   - Store these atoms as raw text.
   - Why raw text? Well, theoretically, we could get structured JSON right away from powerful LLMs. But I've found that focusing on one specific task at a time helps the LLM do a better job. So we're keeping it simple here - just raw text extraction.

3. **Structuring the Atoms**
   - Now we take that raw atom text and turn it into structured data.
   - For this step, we use a less powerful but still decent model (like GPT-3.5-turbo).
   - The idea is that by breaking the process into two steps, we get better quality output. The first LLM focuses solely performing the main task (extracting good atoms), and the second one just has to worry about structuring what's already there.
   - We use a Pydantic model to keep the data structure consistent:
     ```python
     class Atom(BaseModel):
         subject: str
         topic: str
         entities: list[str]
         text: str

     class SpecializedTerminology(BaseModel):
         term: str
         definition: str

     class Atoms(BaseModel):
         atoms: list[Atom]
         specialized_terminologies: list[SpecializedTerminology]
     ```
   - We use the Instructor library with our LLM to pull this structured data from the raw text.

4. **Storing the Data**
   - Use JSONL format to store processed data. It's simple and easy to append to or read from.
   - Write functions to add new data and load existing stuff.

5. **Iterative Processing**
   - Process chunks one after another, using previously processed chunks as examples for consistency.
   - Here's the special sauce: We use a rolling window of the last N chunks plus the chapter/book summary in the prompt. This way, we don't lose context even when processing later parts of the text.

## Some Notes on Implementation

- We're using Google Colab, which has a habit of shutting down. To avoid losing progress, we're storing in-progress data on Google Drive.
- We're using JSONL as it's dead simple. Later, we'll convert this into a proper dataset and upload it to Hugging Face.

This approach should give us a rich, structured dataset from "Meditations" that really captures Marcus Aurelius' philosophy and thought processes. We can then use these atoms to create all sorts of training tasks for our LLM, aiming to make it think and respond like the old Roman emperor himself.

# Code

## Setup

### Setup dependencies

Setup common dependencies, setup keys and setup clients for Litellm & instructor.

In [ ]:
%%capture
!pip install instructor litellm pydantic anthropic openai groq backoff

import litellm
from litellm import completion
from google.colab import userdata
import backoff
import openai

# os.environ["ANTHROPIC_API_KEY"] = userdata.get('ANTHROPIC_KEY')
os.environ["OPENAI_API_KEY"] = userdata.get('OPEN_AI_KEY')
# os.environ["OPENROUTER_API_KEY"] = userdata.get('OPENROUTER_KEY')
# os.environ['GROQ_API_KEY'] = userdata.get('GROQ_KEY')
# os.environ['DEEPSEEK_API_KEY'] = userdata.get('DEEPSEEK_AJ')
# os.environ['DEEPINFRA_API_KEY'] = userdata.get('DEEPINFRA_KEY')

@backoff.on_exception(backoff.expo, openai.RateLimitError)
def gen_from_messages(messages, model='gpt-4o'):
    return completion(
        # model='groq/llama3-70b-8192',
        # model='deepseek/deepseek-chat',
        # model='groq/llama3-70b-8192',
        # model='openrouter/anthropic/claude-3-haiku',
        # model='gpt-3.5-turbo',
        # model='claude-3-haiku-20240307', #'groq/llama3-8b-8192',
        model=model,
        max_tokens=2048,
        messages=messages
    )['choices'][0]['message']['content']

instructor_client = instructor.from_litellm(completion)

### Seed Data


In [ ]:
SUMMARIES = [
    # Book 1
    '''\
Marcus Aurelius’ "Meditations" reflects on the virtues and wisdom he gleaned from various influential figures throughout his life, shaping his Stoic philosophy. Here’s a detailed and thematically organized summary:

#### Family and Early Influences:
1. **Grandfather Verus**: Taught good morals, gentleness, and control over temper.
2. **Father (Annius Verus)**: Exemplified modesty, manly character, and a strong sense of personal integrity.
3. **Mother (Domitia Calvilla/Lucilia)**: Instilled piety, beneficence, generosity, and simplicity in living, avoiding even evil thoughts.
4. **Great-grandfather (Catilius Severus)**: Emphasized the value of private education and good teachers.
5. **Adoptive father (Antoninus Pius)**: Modeled dignity, mildness, perseverance, effective administration, avoidance of petty politics, and a balanced approach to life.

#### Mentors and Teachers:
6. **Diognetus**: Encouraged skepticism of superstition and trivial pursuits, fostering a philosophical mindset and endurance of free speech.
7. **Rusticus (Q. Junius Rusticus)**: Introduced important philosophical texts (Epictetus), encouraged self-improvement, deep understanding of texts, simplicity, forgiveness, and avoidance of public grandstanding.
8. **Apollonius of Chalcis**: Illustrated true freedom, rational living, consistency in adversity, resilience, and gracious acceptance of favours.
9. **Sextus of Chæronea**: Exemplified benevolence, mildness, natural living, tolerance, and unostentatious knowledge, deeply influencing Aurelius' leadership style.
10. **Maximus (Claudius Maximus)**: Demonstrated self-government, cheerfulness, integrity, reliability, and a balance of humour and beneficence.

#### Philosophical Insights:
11. **Ethics and Leadership**: Advocated for justice, moderation, service without desire for accolades, patience, resilience, sincerity, and equanimity.
12. **Stoic Principles**: Emphasized living in harmony with nature, being reflective, maintaining moral integrity, and being unaffected by external validations.

#### Additional Influences:
13. **Alexander the Grammarian**: Promoted constructive criticism, refraining from faultfinding, and effective communication.
14. **Fronto (M. Cornelius Fronto)**: Exposed envy, duplicity, and hypocrisy, critiquing Patricians' lack of paternal affection.
15. **Alexander the Platonic**: Encouraged avoiding unnecessary excuses, prioritizing duties and relationships.
16. **Catulus (Cinna Catulus)**: Advocated responsiveness to friends' criticism, honouring teachers, and loving one's children.
17. **Brother Severus (Claudius Severus)**: Exemplified love for kin, truth, justice, consistency in philosophical practice, and transparency in opinions.

#### Personal Reflections:
- **Personal Growth**: Reflects on his journey of not being swayed by external validations, focusing on genuine relationships and self-mastery.
- **Gratitude and Humility**: Stresses the importance of gratitude, humility, and commitment to personal and ethical growth, recognizing the contributions of his family, mentors, and the gods in shaping his virtues and wisdom.

#### Divine Influence:
- **The Gods**: Expresses gratitude for having good relatives, teachers, associates, and friends, avoiding offenses, personal development, and virtues. Recognizes the divine role in preventing offenses and fostering personal growth.

Through these reflections, Marcus Aurelius provides guidance on leading a life of virtue, emphasizing self-discipline, humility, and the pursuit of wisdom as core to his philosophy and governance. This integrated summary combines specific lessons from individuals with overarching themes, offering a comprehensive view of his Stoic philosophy and personal development.''',


    # Book 2
    '''\
In Book II of "Meditations," Marcus Aurelius delves into the practice of Stoic philosophy, emphasizing self-reflection, rationality, and acceptance of nature's order.

#### Self-Reflection and Inner Life
Marcus begins his day by preparing himself to encounter various negative human behaviors such as arrogance, deceit, and envy. He acknowledges that these traits stem from ignorance of good and evil. By recognizing that all humans share the same divine intelligence, he resolves not to be harmed or angered by others. Instead, he focuses on the importance of internal harmony and the ruling part of his soul, rejecting physical distractions and embracing the transience of life.

#### Human Nature and Cooperation
Marcus emphasizes the inherent kinship among all humans, advocating for natural cooperation rather than conflict. He draws analogies to body parts working together, suggesting that acting against one another is contrary to nature. This understanding guides him to maintain patience and compassion, viewing others' wrongdoings as results of their ignorance.

#### The Divine and Providence
He reflects on the role of Providence, seeing all events as interconnected and part of a greater natural order. Marcus stresses that what happens is for the advantage of the whole universe, including himself. Accepting fate with gratitude, he discards the pursuit of trivial knowledge, focusing instead on living a life aligned with nature and divine will.

#### Philosophical Principles
Central to Marcus' reflections is the Stoic distinction between good and evil, based on reason and virtue. He commits to living each moment with dignity, affection, freedom, and justice. This involves rejecting hypocrisy, self-love, and discontent, and embracing simplicity and sincerity in his actions.

#### Mortality and Life’s Impermanence
Marcus contemplates the inevitability of death and the transient nature of life, reminding himself that all things, including fame and pleasure, are fleeting and insignificant. He views death as a natural process and encourages accepting it without fear. This perspective helps him focus on the present and live in accordance with his philosophical principles.

#### Practical Ethics and Daily Conduct
He advises living each day as if it were the last, performing every action with purpose and without carelessness. By doing so, Marcus aims to achieve a life of tranquility and fulfillment, akin to the existence of the gods, who require nothing more from him than adherence to these few principles.

#### Mental Discipline and Focus
Marcus underscores the importance of mental discipline, avoiding distractions and aimless pursuits. He stresses observing and understanding one's own mind, maintaining the inner daemon (rational self) pure from passions, thoughtlessness, and dissatisfaction. This inner focus ensures that his actions are deliberate and aligned with reason.

#### Philosophical Insights from Others
He references Theophrastus, who argued that offenses driven by desire are more blameworthy than those driven by anger, due to the lack of rational control in the former. Marcus reflects on this insight, recognizing the importance of self-control and rationality in moral conduct. He also acknowledges the influence of Cynic Monimus on the notion that all is opinion, highlighting the subjective nature of human perceptions.

#### Natural Order and Acceptance
Marcus reiterates the necessity of aligning with the natural order, accepting events as they occur. He views the universe as a rational entity that does not err in its administration, and he sees no reason to fear change or dissolution, as these are natural and inevitable processes. By accepting his place within the greater whole, Marcus finds peace and purpose.

### Overall Themes

1. **Stoic Philosophy**: Emphasizing rationality, virtue, and acceptance of nature’s course.
2. **Human Interactions**: Understanding and compassion towards others, recognizing shared humanity and natural cooperation.
3. **Mortality and Impermanence**: Contemplation of death and the transient nature of life to maintain perspective.
4. **Providence and Divine Order**: Trust in the universe's rational order and the providence of the gods.
5. **Self-Improvement and Discipline**: Continuous effort to refine one's character and live according to reason and virtue.
6. **Practical Wisdom**: Applying philosophical insights to daily conduct for a tranquil life.

Through these reflections, Marcus Aurelius provides a guide to living a life of virtue, emphasizing self-discipline, humility, and the pursuit of wisdom as core to his philosophy and governance. This comprehensive summary integrates specific lessons with broader themes, offering a holistic view of his Stoic practices.''',

    # Book 3
    '''\
In Book III of "Meditations," Marcus Aurelius continues to explore Stoic philosophy, emphasizing the impermanence of life, the importance of rationality, and living in harmony with nature.

#### Human Mortality and Impermanence
Marcus reflects on the inevitability of death and the constant wasting away of life. He stresses the urgency of living wisely, as mental faculties may decline with age, making it harder to comprehend and act according to reason. This reflection underlines the necessity of disciplined reason for living a purposeful life and highlights the transient nature of human existence.

#### Natural Beauty and Acceptance
Marcus finds beauty in natural processes and imperfections. He describes the attractiveness in the splitting of baked bread, the ripeness of fruits, and the aging of humans. By observing these natural occurrences, he emphasizes accepting and finding pleasure in the inherent value of all stages and forms of life. This appreciation of natural beauty aligns with the Stoic acceptance of the universe's order.

#### Historical Perspective
Reflecting on historical figures like Hippocrates, Alexander, Pompeius, Caesar, Heraclitus, Democritus, and Socrates, Marcus underscores the certainty of death for all, regardless of their achievements. This historical perspective reminds him that life is fleeting and that human accomplishments, no matter how grand, are ultimately transient.

#### Purposeful Living
Marcus advocates for living with purpose and clarity, avoiding thoughts and actions that do not contribute to the common good. He emphasizes the importance of maintaining a benevolent and socially fitting mindset, ensuring that one's actions and thoughts are clear, honest, and purposeful. This focus on purposeful living is essential for achieving a life of tranquility and fulfillment.

#### Virtue and Rationality
Central to Marcus' reflections is the importance of virtues such as justice, truth, temperance, and fortitude. He finds self-satisfaction through right reason and acceptance of his condition. By keeping the inner deity (rational mind) pure and free from distractions, he maintains a life aligned with Stoic principles, prioritizing rationality and virtue over external validations.

#### Role of the Inner Deity
Marcus reveres the faculty that produces opinions, maintaining freedom from hasty judgments and practicing obedience to the gods. He emphasizes preserving the rational mind (inner deity) uncorrupted and free from passions, thoughtlessness, and dissatisfaction. This inner focus ensures that his actions are deliberate and aligned with reason.

#### Simplicity and Contentment
Marcus embraces simplicity, avoiding complexity and excessive attachment to external validations. He focuses on what is within his control, finding contentment in the present moment. By rejecting distractions and maintaining a simple life, he aligns his actions with his philosophical principles and finds inner peace.

#### Philosophical Preparedness
Marcus emphasizes the importance of being prepared with principles for understanding divine and human matters. He maintains a reference to the bond between the divine and human in every action, ensuring that his actions are consistent with higher values. This philosophical readiness helps him navigate life's challenges with clarity and purpose.

#### Living According to Nature
Aligning actions with natural law and rationality is a key theme in Marcus' reflections. He advocates treating others with benevolence and justice based on natural kinship and accepting fate and the natural course of events without resentment. This acceptance of nature's order is central to Stoic philosophy and guides his actions and thoughts.

#### Mind-Body Distinction
Marcus distinguishes between the body (sensations), soul (appetites), and intelligence (principles). He focuses on what is unique to the good man: contentment with fate and maintaining internal tranquility. By keeping the rational mind pure and uncorrupted, he upholds the core Stoic belief in living a life governed by reason and virtue.

### Broader Themes

1. **Mortality and Impermanence**: Reflecting on the fleeting nature of life and the certainty of death, emphasizing the importance of living wisely and purposefully while mental faculties are intact.
2. **Natural Beauty and Acceptance**: Finding beauty and acceptance in natural processes and imperfections, appreciating the inherent value in all stages and forms of life.
3. **Historical Perspective**: Understanding the inevitability of death for all, including historical figures, which underscores the transient nature of human achievements and existence.
4. **Purposeful Living**: Focusing on thoughts and actions that serve a common good, maintaining clarity, honesty, and purpose in daily conduct.
5. **Virtue and Rationality**: Prioritizing virtues like justice, truth, temperance, and fortitude, and achieving self-satisfaction through rational living and acceptance of one’s circumstances.
6. **Inner Deity and Rational Mind**: Revering and preserving the rational mind (inner deity) free from corruption, hasty judgments, and external influences, emphasizing the divine aspect within.
7. **Simplicity and Contentment**: Embracing simplicity, avoiding distractions, and finding contentment in what is within one’s control, focusing on the present moment.
8. **Philosophical Preparedness**: Always being ready with principles to understand and act in accordance with the divine and human bond, maintaining a reference to higher values in every action.
9. **Living According to Nature**: Aligning actions with natural law, treating others with benevolence and justice, and accepting the natural course of events as part of a greater rational order.
10. **Mind-Body Distinction**: Recognizing the different roles of body, soul, and intelligence, and focusing on maintaining the purity and tranquility of the rational mind, which defines the good man’s life.

Marcus Aurelius provides a comprehensive guide to living a life of virtue, rationality, and acceptance, rooted in Stoic philosophy and a deep understanding of human nature and the natural world.''',

    # Book 4
    '''\
In Book IV of "Meditations," Marcus Aurelius delves deeper into Stoic philosophy, focusing on inner strength, purposeful living, acceptance of fate, and the transient nature of life.

#### Inner Strength and Adaptability
Marcus begins by emphasizing the importance of the inner ruling part (soul/mind) adapting naturally to external events. He uses the analogy of a fire that consumes obstacles to grow stronger, illustrating how challenges can be turned into opportunities for inner growth and resilience.

#### Purposeful and Principled Living
Every action, according to Marcus, should be done with a clear purpose and in accordance with Stoic principles. He insists that one should avoid aimless activities and ensure that all deeds are meaningful and align with virtue and rationality.

#### Internal Retreat and Tranquility
Marcus advises finding peace and renewal within oneself rather than seeking external retreats. He emphasizes that true tranquility comes from the good ordering of the mind and regular self-reflection. By retreating into one's own soul, one can achieve perfect tranquility and cleanse oneself of discontent.

#### Acceptance of Fate and Discontent
Marcus reflects on the nature of human interactions and the importance of accepting one's fate. He argues that rational beings are made for mutual benefit and justice, and enduring wrongs with understanding and patience is part of this natural order. Accepting fate, whether governed by providence or natural chance, leads to inner peace.

#### Transient Nature of Fame and Recognition
Marcus addresses the fleeting nature of fame and external praise, highlighting their insignificance. He notes how quickly things are forgotten and how small the sphere of influence is, urging one to find contentment in the present rather than seeking external validation.

#### Universal Reason and Common Law
Emphasizing shared rationality, Marcus views the world as a political community where all rational beings are fellow citizens governed by a common law. This perspective reinforces the importance of acting in accordance with universal reason and justice.

#### Natural Processes and Death
Death, Marcus notes, is a natural process akin to birth. He encourages acceptance of death as a part of nature’s cycle and not something to be ashamed of or feared. Understanding death as a transformation helps in reconciling with the inevitable end.

#### Opinion and Perception of Harm
Marcus teaches that harm comes not from external events but from one's own opinions. By removing harmful opinions, one can eliminate feelings of being harmed. This Stoic principle emphasizes control over internal judgments rather than external circumstances.

#### Justice, Goodness, and Rational Action
In every action, Marcus advises adhering to justice and goodness. Acting rationally and ethically benefits both the individual and society. He emphasizes that one should always seek to act in ways that are socially and morally beneficial.

#### Philosophical Readiness and Rationality
Maintaining a state of philosophical readiness is crucial. Marcus stresses the importance of being prepared with principles to handle any situation, ensuring that actions are guided by reason and tranquility. This readiness helps navigate life’s challenges with wisdom and calm.

#### Impermanence and Acceptance of Change
Marcus reflects on the impermanence of life and the natural process of change. He encourages viewing life as a continuous flow and accepting change as part of the universe’s nature. This perspective helps in understanding and embracing the transient nature of existence.

#### Self-Contentment and Simplicity
Finding contentment in one’s own actions and nature is essential for inner peace. Marcus advocates for simplicity, avoiding unnecessary thoughts and actions, and focusing on what is within one’s control. This approach leads to a tranquil and fulfilling life.

#### Eternal Perspective and Humility
Marcus frequently contemplates the vastness of time and the insignificance of individual achievements. He reflects on the brevity of life and the countless individuals who have come and gone, emphasizing humility and the importance of focusing on just and social actions.

### Broader Themes

1. **Inner Strength and Adaptability**: Emphasizing the importance of adapting to challenges and using them to grow stronger internally.
2. **Purposeful and Principled Living**: Ensuring that every action is purposeful and aligned with Stoic principles.
3. **Internal Retreat and Tranquility**: Finding peace and renewal within oneself through self-reflection and mental order.
4. **Acceptance of Fate and Discontent**: Accepting fate and enduring wrongs with understanding and patience.
5. **Transient Nature of Fame and Recognition**: Recognizing the fleeting nature of fame and finding contentment in the present.
6. **Universal Reason and Common Law**: Viewing the world as a political community governed by common rationality and justice.
7. **Natural Processes and Death**: Accepting death as a natural process and part of the cycle of life.
8. **Opinion and Perception of Harm**: Eliminating harmful opinions to remove feelings of being harmed.
9. **Justice, Goodness, and Rational Action**: Adhering to justice, goodness, and rationality in all actions.
10. **Philosophical Readiness and Rationality**: Being prepared with Stoic principles to handle any situation calmly and wisely.
11. **Impermanence and Acceptance of Change**: Embracing the transient nature of life and the continuous process of change.
12. **Self-Contentment and Simplicity**: Focusing on simplicity and self-contentment for inner peace.
13. **Eternal Perspective and Humility**: Reflecting on the vastness of time and maintaining humility and perspective on one’s place in the universe.

Marcus Aurelius provides a comprehensive guide to living a life of virtue, rationality, and acceptance. His reflections in Book IV emphasize inner strength, purposeful action, acceptance of fate, and a deep understanding of the transient nature of life, all rooted in Stoic philosophy.''',


    # Book 5
    '''\
In Book V of "Meditations," Marcus Aurelius delves into various aspects of Stoic philosophy, focusing on purpose, resilience, social duty, and acceptance of nature’s order. He offers practical advice on how to live a virtuous and rational life, emphasizing inner strength and tranquility.

#### Purpose and Duty
Marcus begins by reminding himself of the importance of rising each morning to perform his duties as a human being. He questions the inclination to seek comfort over action, using the example of nature’s creatures—plants, birds, ants, spiders, and bees—who all diligently perform their roles. He stresses that humans, too, are meant for action and exertion, not mere pleasure. Nature sets boundaries for rest and sustenance, and exceeding these is contrary to our nature. He urges a commitment to our tasks, valuing societal acts as highly as personal endeavors.

#### Social and Moral Responsibility
Marcus highlights the importance of social acts and moral duties. He lists qualities within one’s control—sincerity, gravity, endurance, aversion to pleasure, contentment, benevolence, and frankness—and urges their cultivation. He criticizes those who fail to exert themselves in these areas, emphasizing that these virtues are always within reach and are not hindered by natural incapacity.

#### Resilience and Inner Strength
Marcus teaches resilience by advocating for the immediate dismissal of troublesome impressions, maintaining tranquility by focusing on actions aligned with nature and universal reason. He emphasizes the importance of adhering to one’s nature and the common nature, accepting the transient nature of life and finding strength in rational thought.

#### Unity with the Universe
Reflecting on death, Marcus contemplates the return to the elements—earth and air—from which life arises. He views human life as interconnected with the universe, with death being a natural process of returning to the source. This unity with the universe reinforces the acceptance of nature’s order and the transient nature of existence.

#### Natural Capacities and Qualities
Marcus urges the development of qualities within one's power, such as sincerity, endurance, and benevolence. He criticizes those who indulge in negative behaviors like murmuring, flattery, and restlessness, pointing out that these actions are within one's control and should be avoided.

#### Acts and Recognition
Performing natural acts without seeking recognition is a key theme. Marcus compares human actions to those of a vine producing grapes, emphasizing that good acts should be performed naturally and without seeking applause. This perspective underscores the importance of fulfilling social duties through self-awareness and internal motivation.

#### Divine and Natural Order
Marcus reflects on the divine and natural order, comparing the acceptance of events to accepting medical prescriptions for health. He argues that everything happens for the health of the universe and should be accepted as part of a greater harmony. This acceptance of the natural order helps in maintaining inner peace and understanding the interconnectedness of all things.

#### Philosophical Practice and Contentment
Philosophy, for Marcus, is a means of returning to right principles and finding contentment with outcomes. He advises not to be discouraged by failures but to continuously strive to align actions with rational principles. This practice leads to inner tranquility and adherence to natural requirements.

#### Perspective on Possessions and Morality
Marcus views possessions and status as transient and often held by unworthy individuals. He stresses maintaining moral principles despite external circumstances and recognizing the fleeting nature of material wealth and societal status. This perspective encourages a focus on inner virtues rather than external validations.

#### Internal Governance and Tranquility
Reflecting on the state of the soul, Marcus emphasizes maintaining calmness and reason in the face of external challenges. He advocates for internal reflection and the cultivation of a tranquil mind that is not disturbed by bodily sensations or external events.

#### Human Nature and Social Interaction
Acting in a social manner and enduring others is a recurrent theme. Marcus advises transforming hindrances into aids for progress, seeing obstacles as opportunities for growth. This approach reinforces the importance of social responsibility and resilience in the face of challenges.

#### Reverence for Rationality
Respecting the rational part within oneself and the universe is crucial. Marcus emphasizes aligning personal actions with universal reason and maintaining reverence for the rational faculty that guides both individual and collective actions.

#### Impermanence and Flux
Observing the constant flow and change of things, Marcus reflects on the transient nature of human affairs. He advises accepting destiny and understanding the impermanence of life, finding peace in the natural dissolution of all things.

#### Acceptance of Destiny
Marcus teaches acceptance of events as part of destiny, encouraging a peaceful and resigned attitude towards life’s outcomes. This acceptance leads to tranquility and a deeper understanding of the natural order.

#### Virtues and Societal Contribution
Practicing virtues like fidelity, modesty, justice, and truth is emphasized. Marcus advises contributing to society without seeking fame or recognition, valuing social acts as expressions of virtue and rationality.

#### Philosophical Detachment
Detaching from bodily sensations and maintaining rational judgment is crucial. Marcus encourages living harmoniously with one’s daemon (inner rational guide) and not being swayed by external circumstances.

#### Gratitude and Reflection
Reflecting on one’s conduct towards others, Marcus expresses gratitude for experiences and the ability to endure and despise pleasures and pains. This reflection fosters humility and appreciation for life’s lessons.

#### Human Ignorance and Learning
Recognizing the ignorance of unskilled souls, Marcus values knowledge and reason. He embraces philosophical learning as a means of overcoming ignorance and achieving inner peace.

#### Finality and Tranquility
Accepting the inevitability of death and finding tranquility in the remaining time is a key theme. Marcus advises appreciating life’s impermanence and living in accordance with nature and reason.

### Broader Themes

1. **Purpose and Duty**: Emphasizing the importance of rising to perform human duties, comparing human work to nature’s creatures, and accepting nature’s limits on rest and sustenance.
2. **Social and Moral Responsibility**: Highlighting the value of social acts and moral duties, and the importance of qualities like sincerity, endurance, and benevolence.
3. **Resilience and Inner Strength**: Teaching resilience by wiping away troublesome impressions and adhering to one's nature and universal nature.
4. **Unity with the Universe**: Reflecting on returning to the elements upon death, emphasizing unity with the universe and natural order.
5. **Natural Capacities and Qualities**: Focusing on the qualities within one’s power and avoiding negative behaviors like murmuring and flattery.
6. **Acts and Recognition**: Performing natural acts without seeking recognition and fulfilling social duties through self-awareness.
7. **Divine and Natural Order**: Accepting events as part of the universal harmony and natural order, and understanding the role of prayer.
8. **Philosophical Practice and Contentment**: Using philosophy to return to right principles, finding contentment with outcomes, and maintaining inner tranquility.
9. **Perspective on Possessions and Morality**: Viewing possessions and status as transient, maintaining moral principles, and recognizing the fleeting nature of external circumstances.
10. **Internal Governance and Tranquility**: Reflecting on the state of the soul, maintaining calmness, and using reason to face external challenges.
11. **Human Nature and Social Interaction**: Acting socially, enduring others, and transforming hindrances into aids for progress.
12. **Reverence for Rationality**: Respecting the rational part within oneself and aligning personal actions with universal reason.
13. **Impermanence and Flux**: Observing the constant change of things, reflecting on the transient nature of human affairs, and accepting destiny.
14. **Virtues and Societal Contribution**: Practicing virtues, contributing to society without seeking fame, and valuing fidelity, modesty, justice, and truth.
15. **Philosophical Detachment**: Detaching from bodily sensations, maintaining rational judgment, and living harmoniously with one’s daemon.
16. **Gratitude and Reflection**: Reflecting on conduct towards others, expressing gratitude for experiences, and recognizing despised pleasures and pains.
17. **Human Ignorance and Learning**: Recognizing ignorance, valuing knowledge and reason, and embracing philosophical learning.
18. **Finality and Tranquility**: Accepting the inevitability of death and finding tranquility in the remaining time, appreciating life’s impermanence.

Marcus Aurelius provides a comprehensive guide to living a life of virtue, rationality, and acceptance in Book V. His reflections emphasize the importance of purpose, resilience, unity with the universe, social responsibility, and inner tranquility, all rooted in Stoic philosophy.
''',


    # Book 6
    '''\
**Book VI of Marcus Aurelius' "Meditations" delves into various aspects of Stoic philosophy, focusing on the nature of the universe, human duty, and the pursuit of virtue. Below is a comprehensive summary that combines detailed analysis with broader themes.**

1. **Nature of the Universe**:
   - The universe is governed by a rational principle that is compliant and devoid of malice. Everything is made and perfected according to this reason, ensuring that nothing is inherently harmful.

2. **Human Duty and Resilience**:
   - Individuals should perform their duties regardless of external conditions such as cold, heat, or fatigue. Whether praised or criticized, the focus should remain on doing what is right.
   - Resilience is emphasized through the metaphor of returning to one's inner harmony after being disturbed, akin to music returning to its key.

3. **Philosophical Reflection**:
   - Constant self-examination and understanding of one's thoughts and actions are crucial. This involves looking within and acknowledging the true nature and value of things, stripping away superficial impressions.

4. **Social Responsibility and Interaction**:
   - Humans are social animals, and their actions should reflect this by contributing to the common good. The importance of acting in a way that is beneficial to society is highlighted, along with the idea of mutual cooperation and harmony.

5. **Acceptance of Fate and Transience**:
   - Life is transient, and everything changes or will eventually be reduced to its basic elements. Accepting this natural process and understanding that nothing lasts forever brings tranquility.
   - Death is seen as a natural act, and one should strive to perform it well, just as one does any other act of life.

6. **Detachment from Fame and External Validation**:
   - The pursuit of fame and public admiration is futile and transient. True value lies in acting according to reason and virtue, rather than seeking validation from others.

7. **Contentment and Simplicity**:
   - Contentment arises from fulfilling one's natural duties and living simply. This involves valuing one's own mind and being in harmony with society and the gods.
   - Simplicity in living and acting according to one's nature is emphasized, rejecting unnecessary desires and external validations.

8. **Virtue and Moral Integrity**:
   - The development of virtues such as justice, wisdom, and fortitude is paramount. Individuals should strive to be sincere, grave, and benevolent, avoiding deceit and embracing a life of virtue.
   - The idea of not becoming like the wrongdoer is highlighted, advocating for the best way of avenging oneself is to remain virtuous.

9. **Interconnectedness of All Things**:
   - The universe is seen as a cohesive and interconnected entity where everything plays a part in the greater whole. Understanding this interconnectedness helps in appreciating the unity and order of the cosmos.

10. **Philosophical Education and Self-Improvement**:
    - Regular learning and philosophical reflection are essential for self-improvement. This involves understanding the nature of things and acting in accordance with reason and virtue.

11. **Perception and Reality**:
    - The way we perceive things significantly affects our experience of them. By adjusting our perceptions to align with reality and reason, we can maintain tranquility and clarity in our actions.

12. **Practical Wisdom and Action**:
    - Wisdom involves practical action that aligns with one's nature and rationality. This includes performing social duties, acting with justice, and maintaining a benevolent disposition towards others.

**Conclusion**:
Book VI of "Meditations" provides a rich exploration of Stoic philosophy, emphasizing the importance of rationality, virtue, and acceptance of the natural order. Marcus Aurelius advocates for a life of simplicity, contentment, and moral integrity, guided by the understanding of the universe's interconnectedness and the transient nature of existence. Through philosophical reflection and self-improvement, individuals can achieve tranquility and fulfill their duties as rational and social beings.
''',


    # Book 7
    '''\
**Book VII** of Marcus Aurelius's "Meditations" focuses on the Stoic principles of rationality, virtue, and acceptance of the natural order. Marcus Aurelius reflects on the transient nature of life, the importance of maintaining inner tranquility, and the interconnectedness of all things in the universe. He emphasizes the following key themes and ideas:

1. **Impermanence and Familiarity**:
   - Life is filled with recurring events and experiences. Nothing is new; everything has been seen before in various forms throughout history. Recognizing this can help one remain undisturbed by life's events.

2. **Rationality and Inner Strength**:
   - One's principles remain alive as long as the corresponding thoughts are maintained. It is within one's power to revive these thoughts and maintain a rational perspective.
   - External events do not affect the mind unless the mind allows them to. Therefore, one should strive to keep the mind undisturbed and focused on rational thoughts.

3. **Acceptance of Change**:
   - Change is a fundamental aspect of nature. Just as physical processes like respiration and digestion involve constant change, so too must humans accept and adapt to changes in life.
   - Embracing change is necessary for growth and harmony with the universal nature.

4. **Social Responsibility and Virtue**:
   - Humans are social animals, meant to live in cooperation with others. Acting for the common good and maintaining social virtues like justice and beneficence is essential.
   - One should not be ashamed to seek help when needed, as cooperation is a part of human nature.

5. **Tranquility and Self-Reflection**:
   - Maintaining inner tranquility involves focusing on one's own actions and judgments rather than being disturbed by external events or others' opinions.
   - Self-reflection and returning to one’s core principles help in achieving and maintaining tranquility.

6. **Contemplation of Mortality**:
   - Death is a natural part of life, akin to other processes of change. It should not be feared but accepted as part of the universal order.
   - Reflecting on the inevitability and naturalness of death helps in putting life's events into perspective and reduces fear and anxiety.

7. **Universal Interconnectedness**:
   - All things are interconnected and part of a greater whole. Understanding this interconnectedness fosters a sense of unity and purpose.
   - The universe is a rational order, and everything within it, including individual actions, contributes to the harmony of the whole.

8. **Simplicity and Modesty**:
   - Simplicity and modesty are virtues to be cultivated. One should avoid unnecessary desires and focus on what is essential and within one’s control.
   - Overvaluing external things leads to disturbance; true contentment comes from valuing inner virtues.

9. **Philosophical Resilience**:
   - Stoic resilience involves maintaining one’s principles and rationality in the face of adversity. This includes not being swayed by pain, pleasure, or the opinions of others.
   - It is important to act justly and follow reason, regardless of external circumstances.

10. **Emulation of Virtuous Examples**:
    - Reflecting on and emulating the virtues of others can provide guidance and inspiration. Observing the virtues in others reinforces one’s own commitment to living a virtuous life.

11. **Practical Wisdom**:
    - Philosophy should be applied in daily life. The practice of philosophy involves rational judgment, ethical behavior, and the pursuit of tranquility.
    - One should live each day as if it were the last, focusing on virtuous actions and maintaining a balanced mind.

In summary, Book VII of "Meditations" by Marcus Aurelius emphasizes the Stoic ideals of rationality, acceptance of nature's order, social responsibility, and inner tranquility. By maintaining a rational perspective, embracing change, acting virtuously, and reflecting on the interconnectedness of all things, individuals can achieve true contentment and harmony with the universe.
''',


    # Book 8
    '''\
**Book VIII of "Meditations" by Marcus Aurelius delves into the intricacies of Stoic philosophy, emphasizing the transient nature of life, the importance of rationality, and the pursuit of virtue.**

**Themes and Detailed Reflections:**

1. **Stoic Philosophy and Rational Living:**
   - Marcus reiterates the core Stoic belief that living according to nature and reason is essential. He emphasizes the futility of seeking fame and the need to focus on the present, acting justly, modestly, and without hypocrisy. Rationality is highlighted as the tool for maintaining inner tranquility and overcoming external disturbances.

2. **Impermanence and Acceptance:**
   - The transient nature of all things is a recurring theme. Marcus reflects on the inevitability of change and death, encouraging acceptance of this natural order. Historical figures, no matter how significant, eventually fade into obscurity, demonstrating the ephemeral nature of fame and legacy.

3. **Duty and Moral Responsibility:**
   - Fulfilling one's duties as a rational and social being is emphasized. Marcus stresses the importance of aligning actions with human nature and the universal law. This includes caring for others, even those who act ungratefully or irrationally, and performing social acts that contribute to the common good.

4. **Self-Reflection and Improvement:**
   - Continuous self-reflection and application of principles from physics, ethics, and dialectics are advised. Marcus underscores the importance of changing one's opinion when corrected and learning from others. This ongoing process of self-improvement is crucial for personal growth and moral development.

5. **Inner Peace and Tranquility:**
   - Maintaining inner peace by focusing on the present and not being disturbed by external events is central. Marcus advises detachment from the desire for external validation and material possessions. The mind's power to remain unaffected by pain and pleasure is a key aspect of achieving tranquility.

6. **Social Relations and Benevolence:**
   - The interconnectedness of all human beings is a significant theme. Marcus advocates for understanding and benevolence towards others, recognizing that everyone is part of a larger whole. Social harmony and cooperation are essential for fulfilling one's purpose and contributing to the common good.

7. **Natural Order and Purpose:**
   - Understanding and accepting the natural order of the universe is crucial. Marcus encourages seeing oneself as part of the universal nature and fulfilling one's role within it. This includes recognizing that every being has a purpose and contributing positively to the universal order.

8. **Practical Wisdom and Daily Life:**
   - Applying philosophical wisdom to everyday situations is emphasized. Marcus likens the art of living to the skills of a wrestler, ready to meet life's challenges. Simplicity, modesty, and contentment with what one has are highlighted as virtues.

9. **Mortality and Legacy:**
   - Reflecting on mortality and the legacy of individuals is a central theme. Marcus contemplates the mortality of famous figures and the impermanence of legacies, encouraging acceptance of death as a natural part of life.

10. **Virtue and Moral Character:**
    - The importance of virtues such as justice, temperance, and benevolence is underscored. Marcus advises striving to maintain a sound mind and moral character in all actions, emphasizing that true satisfaction comes from fulfilling one's duties and acting in accordance with virtue.

11. **Mind and Body:**
    - The distinction between the rational mind and the physical body is explored. Marcus highlights the body's role in experiencing pain and pleasure, while emphasizing the mind's power to remain unaffected and maintain serenity.

12. **Detachment and Focus:**
    - Detachment from the desire for external validation and material possessions is encouraged. Marcus advises focusing on present tasks and recognizing the intrinsic value of actions.

13. **Universal Nature:**
    - The concept of the universal nature governing all things is a key theme. Marcus emphasizes the individual's role as part of a larger whole and the importance of aligning with this universal order.

In summary, **Book VIII of "Meditations" reflects Marcus Aurelius's deep commitment to Stoic principles, emphasizing rational living, acceptance of the transient nature of life, fulfillment of duties, and the pursuit of inner peace and virtue.** Through detailed reflections and broader themes, Marcus provides guidance on how to navigate life's challenges and live a life of wisdom, contentment, and moral integrity.
''',


    # Book 9
    '''\
**Justice and Impiety:**
Marcus Aurelius begins by asserting that acting unjustly is equivalent to acting impiously. This is because the universal nature has created rational beings to help, not harm, one another. Lying is also considered impious, whether intentional or not, as it goes against the universal nature and truth, which are core to the cosmic order.

**Falsehood and Deception:**
The emperor discusses the consequences of deception, emphasizing that lying disturbs the natural order and harmony. Both intentional and unintentional falsehoods are detrimental as they contradict the universal nature's principles, which are aligned with truth.

**Pleasure and Pain:**
Marcus addresses the Stoic view on pleasure and pain, arguing that pursuing pleasure and avoiding pain often lead to impiety. This pursuit frequently results in dissatisfaction with the natural order, as people tend to blame the universe for perceived injustices, such as bad people enjoying pleasure while good people suffer.

**Nature’s Will and Acceptance of Death:**
He encourages acceptance of life stages and death as natural processes. Just as it is natural for a child to be born, it is natural for a person to die. He stresses the importance of not fearing death but accepting it as a part of nature's design.

**Moral Responsibility:**
Marcus highlights the idea that wrong actions harm the doer most of all. Acting unjustly deteriorates one's own moral character. He emphasizes the importance of aligning actions with social good and maintaining a disposition of contentment.

**Imagination, Desire, and Appetite:**
He advises controlling imagination, desire, and appetite to maintain the ruling faculty's power. This self-control is essential for a tranquil and virtuous life.

**Unity and Interconnection:**
The concept of unity among rational beings is explored. Marcus explains that everything in the universe, from elements to rational beings, is interconnected and naturally inclined to come together. Rational animals (humans) have forgotten this mutual desire and inclination, leading to social discord.

**Rational and Social Acts:**
The importance of rational and socially beneficial actions is emphasized. Rational beings should align their actions with reason and social good, as this is the natural and intended way of living.

**Change and Impermanence:**
Marcus reiterates the constant change in life and the universe, urging acceptance of this impermanence. He suggests reflecting on the rapid changes and the fleeting nature of life to cultivate a sense of detachment and acceptance.

**Judgment and Perception:**
He discusses the role of the ruling faculty in forming judgments and perceptions. The way we perceive and judge external events is within our control and shapes our experience of the world.

**Inner Peace:**
Achieving inner tranquility is possible by aligning oneself with nature and reason. Marcus emphasizes the importance of maintaining a tranquil mind and not being disturbed by external events.

**Social Relations and Tolerance:**
Understanding and bearing with others’ faults is crucial. He encourages patience and gentle correction of others, recognizing that human errors are natural and should be met with compassion.

**Leading Principles:**
Understanding one's own and others' ruling principles is vital. This self-awareness and understanding of others help in dealing with social interactions and maintaining harmony.

**External vs. Internal Causes:**
He distinguishes between disturbances caused by external factors and those within our control. Inner peace comes from focusing on what we can control and accepting external events as they are.

**Fruitfulness:**
Marcus uses the metaphor of producing fruit to describe the outcomes of reason and action. Just as trees bear fruit, rational beings produce results through their thoughts and actions.

**Indulgence and Correction:**
The role of indulgence and correction in dealing with others’ wrongdoings is highlighted. Marcus advises correcting others gently and understandingly, just as the gods are indulgent towards human faults.

**Action vs. Passivity:**
He believes in the importance of active engagement in life rather than passive existence. Virtue lies in activity and purposeful action aligned with reason and nature.

**Cosmic Perspective:**
Viewing life from a broader cosmic perspective helps in understanding its transience and insignificance of fame and external validation. This perspective fosters detachment and acceptance of the fleeting nature of life.

**Human Nature:**
Marcus reflects on human behavior, including the inevitability of faults and the necessity of tolerance. He urges understanding and compassion towards others, recognizing that everyone has their own struggles.

**Cosmic Order and Natural Laws:**
The universe operates according to natural laws and order. Aligning with these laws leads to a harmonious life. Marcus stresses that everything happens according to a greater plan, and accepting this brings tranquility.

**Active Engagement vs. Passivity:**
Active engagement in life, focusing on the present, performing one's duties, and contributing to the common good are crucial. Marcus advises against passivity and encourages active participation in life.

### Broader Themes in Book IX

1. **Interconnectedness and Unity:**
   - Marcus emphasizes the interconnectedness of all rational beings and the universe. He stresses that human beings are meant to support one another and should strive for unity and mutual support.

2. **Impermanence and Acceptance:**
   - The transient nature of life and the inevitability of change and death are recurrent themes. Marcus encourages acceptance of these natural processes and understanding that everything, including suffering and joy, is temporary.

3. **Rationality and Virtue:**
   - Rationality is seen as the guiding principle for a virtuous life. Marcus stresses the importance of rational actions, social good, and aligning oneself with universal nature.

4. **Moral Responsibility and Inner Peace:**
   - Acting unjustly harms oneself, and maintaining moral responsibility is crucial for inner peace. Marcus advises controlling imagination, desires, and maintaining a tranquil mind through rational judgment.

5. **Social Conduct and Tolerance:**
   - Understanding and tolerating the faults of others is essential. Marcus advocates for patience and gentle correction rather than harsh judgment, recognizing that everyone has their own struggles and errors.

6. **Perspective on Life and Death:**
   - Life should be viewed from a cosmic perspective, understanding its brevity and the insignificance of fame and external validation. Death is a natural part of life, and fearing it is irrational.

7. **Self-Improvement and Philosophy:**
   - Continuous self-reflection and adherence to philosophical principles are essential for personal growth. Marcus underscores the importance of philosophy in guiding one's actions and thoughts, even in adversity.

8. **Human Behavior and Nature:**
   - Insights into human nature, including the inevitability of faults and the necessity of tolerance, are explored. Marcus reflects on the behaviors and motivations of people, urging understanding and compassion.

9. **Cosmic Order and Natural Laws:**
   - The universe operates according to natural laws and order, and aligning with these laws leads to a harmonious life. Marcus stresses that everything happens according to a greater plan, and accepting this brings tranquility.

10. **Active Engagement vs. Passivity:**
    - The importance of active engagement in life, rather than passive existence, is emphasized. Marcus advises focusing on the present, performing one's duties, and contributing to the common good.

In summary, **Book IX of "Meditations" explores the interconnectedness of life, the acceptance of impermanence, the importance of rationality and virtue, and the necessity of moral responsibility and inner peace.** Marcus Aurelius provides a comprehensive guide on how to navigate life with wisdom, tolerance, and a cosmic perspective, emphasizing the Stoic principles of self-improvement, social conduct, and alignment with natural laws.
''',


    # Book 10
    '''\
**Living in Accordance with Nature:**
- Marcus Aurelius emphasizes the importance of aligning oneself with nature and rationality. He encourages the soul to be simple, content, and free from unnecessary desires. True satisfaction comes from accepting and appreciating the present, understanding that everything is provided by the gods and is part of the natural order.

**Acceptance and Endurance:**
- One must bear whatever happens with equanimity, recognizing that everything is either within one's capacity to endure or will soon pass. Complaints are futile; instead, one should strive to see challenges as opportunities to exercise virtue.

**Interconnectedness and Social Nature:**
- Humans are social animals, and it is essential to live in harmony with others. This involves contributing to the common good and understanding one's role within the larger whole. Rational beings should act justly, aiming for the benefit of the community.

**Reflection on Mortality and Change:**
- Marcus reflects on the transient nature of life, encouraging acceptance of change and dissolution as natural processes. He compares life to a play, where each person plays their part before exiting. Death is a return to the elements from which one came, and it should be met without fear.

**Maintaining Virtue and Integrity:**
- Adhering to virtues such as simplicity, rationality, and magnanimity is crucial. Marcus advises maintaining these virtues without seeking external validation. Living a virtuous life is about embodying these principles consistently.

**Philosophical Contemplation:**
- Contemplation and philosophical reflection are vital for understanding the nature of things. By constantly examining one's thoughts and actions, one can achieve a state of inner peace and rationality.

**Human Behavior and Judgment:**
- Marcus urges kindness and self-reflection when dealing with others' mistakes. Recognizing that everyone is interconnected, he advocates for patience and understanding. When others err, it is often due to ignorance, and one should respond with compassion rather than anger.

**Practical Stoicism:**
- Stoicism involves acting justly and reasonably, regardless of external circumstances. Marcus emphasizes the importance of focusing on one's own actions and maintaining tranquility. External events are beyond one's control, but how one responds to them is a matter of choice.

**Impermanence of Human Affairs:**
- Human achievements and reputations are fleeting. Marcus advises detachment from the pursuit of fame and external validation, recognizing that all human endeavors are temporary.

**Resilience and Inner Strength:**
- True strength comes from within, from maintaining one's principles and rationality. Even in the face of adversity, one can remain unperturbed by focusing on what is within one's control and acting according to reason.

**Detachment from External Validation:**
- Seeking praise or fearing criticism is unnecessary. Marcus stresses the importance of self-contentment and living according to one's principles without concern for others' opinions.

**Facing Mortality with Tranquility:**
- Death is a natural part of life and should be faced calmly. Marcus encourages seeing it as a return to the universal nature and an end to one's role in the play of life. Acceptance of mortality leads to a more peaceful and fulfilled life.

In **Book X of "Meditations," Marcus Aurelius delves into the principles of Stoicism, emphasizing acceptance, rationality, virtue, and the transient nature of life. He provides guidance on how to live harmoniously with nature and others, maintain inner peace, and face mortality with tranquility.**
''',


    # Book 11
    '''\
**Self-Awareness and the Rational Soul:**
Marcus Aurelius begins by discussing the properties of the rational soul. It has the ability to see and analyze itself, choose its actions, and find fulfillment in its own virtues. The rational soul can enjoy its fruits directly, unlike plants or animals, whose fruits benefit others. It achieves its purpose in every moment, regardless of how long life lasts, making every part of its existence complete.

**Virtue and Self-Contentment:**
He emphasizes that true contentment comes from within and is not dependent on external factors like pleasure, time, place, or companionship. A person should be satisfied with their current condition, recognizing that everything they have comes from the gods and is therefore good. He urges maintaining an affectionate, contented disposition and living in harmony with both gods and humans.

**Philosophical Life:**
Aurelius reflects on the philosophical life, noting that virtue and virtuous actions are the only things of real value. He advises breaking down and analyzing pleasures and desires to see them as insignificant. The readiness to face death calmly and without dramatics, in contrast to Christians who face it with obstinacy, is a mark of true philosophical understanding.

**Interpersonal Relationships and Social Harmony:**
He highlights the importance of social connections and the impact of one's actions on the community. He uses the metaphor of a branch cut off from a tree to illustrate how a person separated from others harms the social body. He advises maintaining benevolent feelings towards others, even those who hinder or trouble you, and striving for unity.

**Nature, Justice, and Universal Order:**
Aurelius asserts that nature and universal order do not fall short of human art; rather, they surpass it. Justice and other virtues are rooted in nature. He advises accepting whatever happens as natural and beneficial for the whole, recognizing that the parts (individuals) are subordinate to the whole (the universe).

**Indifference and Detachment:**
The emperor stresses the importance of indifference to external things. By understanding that external events do not inherently affect one's rational soul, a person can maintain tranquility. He advises detachment from things beyond one's control and focusing on what is within one's power—namely, one's own judgments and actions.

**Self-Reflection and Contemplation:**
Self-reflection and contemplation are crucial for understanding the transient nature of life and the universe. He encourages examining the nature, origin, and eventual dissolution of all things to appreciate their temporary nature. This perspective helps in maintaining clarity and focus on what is truly important.

**Mortality and Acceptance of Fate:**
He reflects on mortality and the naturalness of death, urging acceptance rather than fear. Life should be lived in accordance with nature, recognizing that death is just another change. By contemplating the brevity of life and the inevitability of death, one can find peace and live more fully.

**Moral Integrity and Authenticity:**
Maintaining moral integrity and authenticity is vital. Aurelius advises living in accordance with one's principles and values, regardless of external opinions or pressures. He cautions against false displays of virtue and emphasizes genuine goodness that naturally reveals itself through actions and demeanor.

**Philosophical Practice:**
The practice of philosophy involves continuous self-improvement and adherence to principles. Learning from past philosophers and applying their teachings to present actions is essential. He advocates for simplicity in thought and life, focusing on what is essential and dismissing distractions.

**Human Emotions and Passions:**
Aurelius advises managing emotions and passions through reason and reflection. Anger, grief, and fear are seen as deviations from nature. He emphasizes the importance of maintaining a kind disposition, even towards those who do wrong, and correcting them gently.

**Education and Learning:**
He values education and the role of learning in developing virtue and moral integrity. Drawing on the teachings of philosophers like Socrates and Epictetus, he underscores the importance of continuous self-improvement and philosophical practice.

**Final Reflections:**
Aurelius concludes with reflections on simplicity, the inevitability of change, and the role of human beings in the universe. He urges living in harmony with nature, accepting fate, and striving for social harmony and justice. By maintaining a clear and focused mind, one can achieve true contentment and fulfillment.

In summary, **Book 11** of "Meditations" explores the nature of the rational soul, the pursuit of virtue, the importance of self-awareness and contemplation, the need for social harmony, and acceptance of the natural order and mortality. It emphasizes moral integrity, detachment from externals, and the practice of philosophy as a way to achieve true contentment and live a fulfilling life.
''',


    # Book 12
    '''\
#### Personal Conduct and Virtue
Marcus Aurelius emphasizes the importance of living in the present and acting with piety and justice. He advocates for contentment with one's lot as assigned by nature, and for speaking the truth and acting according to the law. He urges focusing solely on one's ruling faculty (the mind) and the divinity within, neglecting past concerns and trusting the future to providence. Aurelius stresses that fearing never to have lived according to nature is more significant than fearing death.

#### The Nature of Existence
He breaks down human existence into three components: body, breath (life), and intelligence, with intelligence being the most significant. By detaching from past actions, future worries, and external influences, one can live a life true to their intellect, free from perturbations, and obedient to their inner daemon. He admires Empedocles' concept of a sphere, symbolizing self-sufficiency and harmonious rest.

#### God and Providence
Aurelius ponders the role of gods in arranging the universe benevolently and questions why virtuous men do not continue to exist after death if it were truly beneficial. He concludes that the gods, being just, would have arranged things differently if it were right to do so. He encourages practice even in despair, using the example of the left hand becoming more skilled at holding the bridle through practice.

#### Philosophical Practice
He emphasizes continuous self-improvement and adherence to principles, likening the practice to a pancratiast (a complete athlete) rather than a gladiator who might lose his weapon. This includes seeing things as they are by separating them into matter, form, and purpose. Aurelius underscores the importance of acting justly, accepting events as they happen, and maintaining truthfulness.

#### Mortality and Death
Aurelius reflects on the inevitability of death, the shortness of life, and the boundless time before and after one's existence. He contemplates the fleeting nature of fame and pleasure and concludes that everything is transient. He advises viewing death as a natural part of life, focusing instead on living according to reason and virtue.

#### Social Interaction
Maintaining equanimity and kindness in social interactions is crucial. Aurelius advises understanding and compassion towards others, recognizing that wrongful acts stem from ignorance. He stresses the importance of mutual benefit and social duty, urging avoidance of blame and judgment.

#### Philosophical Reflection
Aurelius reflects on the universality of human experiences, the vastness of time, and the insignificance of individual concerns. He emphasizes that all things are opinion-based and within one's power to change. By accepting the natural order and seeing the commonality in human struggles, one can achieve tranquility.

#### Consistency and Self-Improvement
Consistency in goals and actions is essential. Aurelius urges aligning personal conduct with universal nature and being self-aware and self-corrective. He promotes simplicity, contentment, and reducing desires to achieve inner peace.

#### Contentment and Simplicity
Finding contentment in simplicity and embracing a minimalist approach to desires and attachments are emphasized. Aurelius advises focusing on inner peace and harmony rather than external achievements, advocating for a life of virtue and reason.

#### Universal Perspective
Aurelius adopts a universal perspective, understanding the interconnectedness of all things. He reflects on the insignificance of personal concerns within the vastness of the cosmos, urging others to live according to their nature and endure what the common nature brings.

### Broader Themes

1. **Living According to Nature:**
   - Harmonizing with nature and accepting events as natural occurrences.
   - Acting in accordance with one's true self and universal principles.

2. **The Power of the Mind:**
   - The mind's ability to control perceptions and judgments.
   - Focusing on the present and detaching from external influences.

3. **Acceptance and Resignation:**
   - Trusting in divine providence and accepting one's fate.
   - Recognizing the benevolent arrangement of the universe.

4. **Mortality and Transience:**
   - Embracing the transient nature of life and death.
   - Finding peace in the inevitability of change.

5. **Philosophical Practice and Self-Improvement:**
   - Continuous self-improvement through philosophical practice.
   - Adherence to principles and consistency in actions.

6. **Social Duty and Justice:**
   - Acting justly and with kindness in social interactions.
   - Understanding and compassion towards others.

7. **Contentment and Simplicity:**
   - Achieving inner peace through simplicity and reducing desires.
   - Prioritizing virtue and reason over external achievements.

8. **Universal Perspective:**
   - Recognizing the interconnectedness of all things.
   - Viewing personal concerns within the vastness of the cosmos.

In conclusion, Book 12 of "Meditations" encapsulates Marcus Aurelius' Stoic philosophy, stressing the importance of living according to nature, the power of the mind, acceptance of fate, continuous self-improvement, social duty, contentment, simplicity, and adopting a universal perspective. These reflections serve as a guide for achieving tranquility, inner peace, and a life of virtue.
'''
]

SEED = [
    {
        'book_num': 1,
        'para_num': 1,
        'chunk_index': None,
        # Book 1 - Chunk 1
        'atoms_raw': '''\
Subjects:
- Marcus Aurelius
- Grandfather Verus

Topics:
- Moral teachings
- Emotional self-control

Entities:
- Marcus Aurelius
- Grandfather Verus (Annius Verius)

Specialized Terminology: None

Atom 1:
Subject: Marcus Aurelius
Topic: Moral teachings
Entities: Marcus Aurelius, Grandfather Verus (Annius Verius)

Marcus Aurelius, the Roman emperor, learned good morals from his grandfather, Annius Verius. Aurelius attributes his understanding of ethical principles and virtuous behavior to the teachings and influence of his grandfather.

Atom 2:
Subject: Marcus Aurelius
Topic: Emotional self-control
Entities: Marcus Aurelius, Grandfather Verus (Annius Verius)

Marcus Aurelius, the Roman emperor, learned the importance of managing his temper and controlling his emotions from his grandfather, Annius Verius. Aurelius's ability to govern his emotional responses and maintain composure can be attributed to the lessons imparted by his grandfather.''',
    },
    # Book 1 - Chunk 2
    {
        'book_num': 1,
        'para_num': 2,
        'chunk_index': None,
        # Book 1 - Chunk 1
        'atoms_raw':'''\
Subjects:
- Marcus Aurelius
- His father (Annius Verus)

Topics:
- Modesty
- Masculinity

Entities:
- Marcus Aurelius
- Annius Verus (Father)

Specialized Terminology: None

Atom 1:
Subject: Marcus Aurelius
Topic: Modesty
Entities: Marcus Aurelius, Annius Verus (Father)

Marcus Aurelius, the Roman emperor, learned the value of modesty from the reputation and memories associated with his father, Annius Verus. Aurelius credits the humble and restrained behaviors demonstrated and remembered in his father as key influences on his own understanding and practice of modesty.

Atom 2:
Subject: Marcus Aurelius
Topic: Masculinity
Entities: Marcus Aurelius, Annius Verus (Father)

Marcus Aurelius acquired traits of a manly character, which he attributes to the reputation and remembrance of his father, Annius Verus. These traits, reflecting courage, resilience, and a responsible male role, were recognized and absorbed by Aurelius from the narratives and legacy left by his father.''',
    },

    # Book 1 - Chunk 3
    {
        'book_num': 1,
        'para_num': 3,
        'chunk_index': None,
        # Book 1 - Chunk 1
        'atoms_raw':'''\
Subjects:
- Marcus Aurelius
- His mother (Domitia Calvilla)

Topics:
- Piety
- Beneficence
- Abstinence from evil deeds and thoughts
- Simplicity in lifestyle

Entities:
- Marcus Aurelius
- Domitia Calvilla (Mother)

Specialized Terminology: None

Atom 1:
Subject: Marcus Aurelius
Topic: Piety
Entities: Marcus Aurelius, Domitia Calvilla (Mother)

Marcus Aurelius, Roman Emperor, attributes his development of piety to the teachings and examples set by his mother, Domitia Calvilla, also known as Lucilia. Through her influence, he embraced a religious and moral devotion that shaped his personal and philosophical life.

Atom 2:
Subject: Marcus Aurelius
Topic: Beneficence
Entities: Marcus Aurelius, Domitia Calvilla (Mother)

From his mother, Domitia Calvilla, Marcus Aurelius learned the importance of beneficence, which entails the act of doing good and showing kindness to others. This principle became a fundamental aspect of his personal virtues and his leadership as emperor.

Atom 3:
Subject: Marcus Aurelius
Topic: Abstinence from evil deeds and thoughts
Entities: Marcus Aurelius, Domitia Calvilla (Mother)

Marcus Aurelius was taught by his mother, Domitia Calvilla, the importance of abstaining not only from evil deeds but also from evil thoughts. This teaching contributed significantly to his stoic philosophy, emphasizing not just outward actions but also the inner moral discipline.

Atom 4:
Subject: Marcus Aurelius
Topic: Simplicity in lifestyle
Entities: Marcus Aurelius, Domitia Calvilla (Mother)

The Roman Emperor Marcus Aurelius adopted a simple way of living, a trait he credits to the influence of his mother, Domitia Calvilla. She instilled in him the value of leading a life that is distanced from the extravagance typically associated with the wealthy, focusing instead on a more austere and straightforward lifestyle.''',
    },

    # Book 1 - Chunk 4
    {
        'book_num': 1,
        'para_num': 4,
        'chunk_index': None,
        # Book 1 - Chunk 1
        'atoms_raw':'''\
Subjects:
- Marcus Aurelius
- His great-grandfather (Catilius Severus)

Topics:
- Education in private settings
- Importance of good teachers
- Investing liberally in education

Entities:
- Marcus Aurelius
- Catilius Severus (Great-grandfather)

Specialized Terminology: None

Atom 1:
Subject: Marcus Aurelius
Topic: Education in private settings
Entities: Marcus Aurelius, Catilius Severus (Great-grandfather)

Marcus Aurelius, influenced by his great-grandfather Catilius Severus, learned the value of receiving education in private settings rather than attending public schools. This approach allowed him to have a more personalized and focused educational experience during his formative years.

Atom 2:
Subject: Marcus Aurelius
Topic: Importance of good teachers
Entities: Marcus Aurelius, Catilius Severus (Great-grandfather)

Marcus Aurelius was taught by his great-grandfather Catilius Severus the importance of having competent and knowledgeable teachers at home. This principle shaped Aurelius's educational path, ensuring he received instructions from tutors who could effectively contribute to his intellectual and ethical development.

Atom 3:
Subject: Marcus Aurelius
Topic: Investing liberally in education
Entities: Marcus Aurelius, Catilius Severus (Great-grandfather)

From Catilius Severus, his great-grandfather, Marcus Aurelius learned the importance of investing liberally in education. This advice emphasized that resources should be generously allocated to learning, as it is crucial for personal and professional growth.''',
    },

    # Book 1 - Chunk 5
    {
        'book_num': 1,
        'para_num': 5,
        'chunk_index': None,
        # Book 1 - Chunk 1
        'atoms_raw':'''\
Subjects:
- Marcus Aurelius
- His governor

Topics:
- Neutrality in sports and entertainment
- Endurance in labor
- Minimalism and self-sufficiency
- Non-interference in others' affairs
- Avoidance of slander

Entities:
- Marcus Aurelius
- Governor (Unnamed)
- Green and blue parties (Circus games factions)
- Parmularius and Scutarius (Gladiator fight factions)

Specialized Terminology:
- Green and blue parties
- Parmularius
- Scutarius

Definitions:
- **Green and blue parties**: These were factions or teams supported by different groups during chariot races in ancient Rome, often associated with social and political rivalries.
- **Parmularius and Scutarius**: These terms refer to types of gladiators; "Parmularius" being one equipped with a small shield, and "Scutarius" with a large shield, indicating different fighting styles and affiliations in the gladiatorial games.

Atom 1:
Subject: Marcus Aurelius
Topic: Neutrality in sports and entertainment
Entities: Marcus Aurelius, Governor (Unnamed), Green and Blue parties, Parmularius, Scutarius

Marcus Aurelius, mentored by his unnamed governor, learned to maintain neutrality in popular entertainments such as the chariot races of the Circus, where factions were divided into green and blue parties, and gladiator fights, where competitors were categorized as Parmularius or Scutarius. This principle fostered an attitude of impartiality and fairness in his later life and rule.

Atom 2:
Subject: Marcus Aurelius
Topic: Endurance in labor
Entities: Marcus Aurelius, Governor (Unnamed)

From his governor, Marcus Aurelius learned the value of enduring labor. This teaching stressed the importance of persistence and resilience in both physical and mental tasks, which was integral to his practices as a Stoic and a ruler.

Atom 3:
Subject: Marcus Aurelius
Topic: Minimalism and self-sufficiency
Entities: Marcus Aurelius, Governor (Unnamed)

Marcus Aurelius was taught by his governor the principles of wanting little and working with his own hands. This guidance promoted a lifestyle of minimalism and self-sufficiency, contributing to his philosophical outlook of simplicity and independence.

Atom 4:
Subject: Marcus Aurelius
Topic: Non-interference in others' affairs
Entities: Marcus Aurelius, Governor (Unnamed)

Under his governor's mentorship, Marcus Aurelius learned the importance of not meddling in other people's affairs. This lesson encouraged respect for the autonomy of others and helped shape his rule as an emperor, emphasizing non-interference and respect for personal boundaries.

Atom 5:
Subject: Marcus Aurelius
Topic: Avoidance of slander
Entities: Marcus Aurelius, Governor (Unnamed)

Marcus Aurelius was guided by his governor to avoid being receptive to slander. This lesson underlined the significance of avoiding gossip and misinformation, fostering a climate of integrity and truthfulness in his personal and public life.'''
    }
]

### Core Code

#### Download and Process raw data

In [ ]:
import os
import json
from bs4 import BeautifulSoup

def extract_endnotes(xhtml_content):
    soup = BeautifulSoup(xhtml_content, 'html.parser')
    endnotes = []
    for li in soup.find_all('li', {'epub:type': 'endnote'}):
        endnote_id = int(li['id'].split('-')[-1])
        endnote_text = li.get_text(strip=True)
        endnotes.append((endnote_id, endnote_text))

    endnotes.sort(key=lambda x: x[0])
    return [endnote[1] for endnote in endnotes]

def process_xhtml(xhtml_content, book_num):
    soup = BeautifulSoup(xhtml_content, 'html.parser')
    paragraphs = []
    for i, p in enumerate(soup.find_all('p'), start=1):
        chunks = process_paragraph(p, i, book_num)
        paragraphs.extend(chunks)
    return paragraphs

def process_paragraph(paragraph, para_num, book_num):
    max_length = 1500
    text = paragraph.get_text()
    endnotes = []
    for link in paragraph.find_all('a', {'epub:type': 'noteref'}):
        endnote_id = link.get('href').split('#')[-1]
        endnote_num = int(endnote_id.split('-')[-1])
        text = text.replace(link.get_text(), f'[{endnote_num}]')
        endnotes.append(endnote_num)

    if len(text) <= max_length:
        return [{'text': text, 'endnotes': endnotes, 'para_num': para_num, 'book_num': book_num}]

    chunks = []
    current_chunk = ""
    current_endnotes = []
    statements = text.split('.')
    for statement in statements:
        statement = statement.strip()
        if not statement:
            continue
        if len(current_chunk) + len(statement) + 1 <= max_length:
            current_chunk += '. ' + statement if current_chunk else statement
        else:
            if current_chunk:
                chunk_index = len(chunks) + 1
                chunks.append({'text': current_chunk + '.',
                               'endnotes': current_endnotes,
                               'para_num': para_num,
                               'chunk_index': chunk_index,
                               'book_num': book_num})
            current_chunk = statement
            current_endnotes = []

        for endnote_num in endnotes:
            if f'[{endnote_num}]' in statement:
                current_endnotes.append(endnote_num)

    if current_chunk:
        chunk_index = len(chunks) + 1
        chunks.append({'text': current_chunk + '.',
                       'endnotes': current_endnotes,
                       'para_num': para_num,
                       'chunk_index': chunk_index,
                       'book_num': book_num})

    return chunks

def render_chunk(chunk, endnotes_list):
    text = chunk['text']
    endnotes = chunk['endnotes']
    para_num = chunk['para_num']
    book_num = chunk['book_num']
    chunk_index = chunk.get('chunk_index')  # chunk_index may not be present for intact paragraphs

    rendered_text = text + '\n\n'
    for endnote_num in endnotes:
        endnote_index = endnote_num - 1
        if 0 <= endnote_index < len(endnotes_list):
            endnote_text = endnotes_list[endnote_index]
            rendered_text += f'[{endnote_num}]: {endnote_text}\n'

    rendered_chunk = {
        'text': rendered_text.strip(),
        'endnotes': endnotes,
        'para_num': para_num,
        'book_num': book_num
    }
    if chunk_index is not None:
        rendered_chunk['chunk_index'] = chunk_index

    return rendered_chunk

def extract_chunks():
    epub_dir = 'meditations/src/epub/text/'

    # Load raw xml file for meditation ebook
    !curl -L https://github.com/standardebooks/marcus-aurelius_meditations_george-long/archive/refs/heads/master.zip -o meditations_snapshot.zip
    !unzip ./meditations_snapshot.zip -d meditations
    !cp -r meditations/marcus-aurelius_meditations_george-long-master/src meditations/src


    # Open endnotes; we will join it with chapter data
    # so that we have enough context while processing them
    with open(os.path.join(epub_dir, 'endnotes.xhtml'), 'r', encoding='utf-8') as file:
        endnotes_xhtml = file.read()
    endnotes_list = extract_endnotes(endnotes_xhtml)


    # Process each book and break it into paragraphs
    books_paragraphs = []
    for i in range(1, 13):
        book_file = f'book-{i}.xhtml'
        with open(os.path.join(epub_dir, book_file), 'r', encoding='utf-8') as file:
            book_xhtml = file.read()
        book_paragraphs = process_xhtml(book_xhtml, i)
        books_paragraphs.extend(book_paragraphs)

    # Render chunks and store in JSON file
    rendered_books_paragraphs = [render_chunk(chunk, endnotes_list) for chunk in books_paragraphs]

    return rendered_books_paragraphs

#### Extract Atoms from chunk

In [ ]:
import os
import json

SYSTEM_PROMPT_TEMPLATE = '''\
You are an expert knowledge extraction system designed to analyze a book chapter by chapter and chunk by chunk. Your task is to carefully read each text chunk and extract key "atoms" of information.

You have deep knowledge about Roman History, Stoic Philosophy and life of Marcus Aurelius.

For each chunk, perform the following steps:

1. Subjects, Topics, Entities, and Terminology:
   - Identify the main subject or subjects of the text chunk
   - Determine the key topic or topics covered in the chunk
   - List all the entities (people, places, objects, etc.) mentioned in the chunk
   - Identify any specialized terminology used in the chunk

2. Definitions:
   - Provide clear and concise definitions for any specialized terminology identified in the previous step

3. Atom Generation:
   - Create one atom per topic. Each atom should only cover a single topic.
   - Carefully craft each atom as a self-contained unit of information that can be understood independently without relying on context from other atoms or the chapter summary
   - Ensure that each atom includes all necessary details, such as the identification of individuals, their roles, relationships, and any relevant background information
   - Avoid using pronouns or references to other atoms; instead, explicitly include all relevant information within the atom itself
   - Use simple and clear sentence structures to ensure clarity and avoid convoluted phrasing
   - If an atom requires multiple sentences or a paragraph to convey all the necessary information, feel free to write as much as needed to ensure the atom is complete and independent

4. Output:
   - Present the atoms as a numbered list
   - For each atom, first provide the following details:
     - Subject(s)
     - Topic(s)
     - Entities
     - Specialized Terminology (if any)
   - Then, present the atom itself as a complete, self-contained unit of information

Proceed with analyzing the text chunk and extracting the atoms according to the provided structure.

# Details
Title: Meditations
Author: Emperor of Rome Marcus Aurelius
Language: English
Translator: George Long

# Book {book_number} Summary

{chapter_summary}
'''

def append_data_jsonl(data, file_path):
    with open(file_path, 'a') as f:  # 'a' for append mode
        for item in data:
            json.dump(item, f)
            f.write('\n')  # Move to the next line for the next record

def render_chunk_message(chunk):
    chunk_idx_text = f"Part: {chunk['chunk_index']}\n" if 'chunk_index' in chunk else ''
    input_text = f'''\
Book: {chunk['book_num']}
Paragraph: {chunk['para_num']}
{chunk_idx_text}
{chunk['text']}'''
    return input_text

def extract_atoms_as_raw_text(all_chunks, file_path, seed_data, N=5):
    n = min(len(seed_data), N)
    l = len(all_chunks)

    chunks_processed = [(all_chunks[i], x) for i, x in enumerate(SEED)]
    print(len(SEED), len(chunks_processed))
    to_store = SEED
    for i in range(n, l):
        start_idx   = i - n
        end_idx     = i
        examples    = chunks_processed[start_idx:end_idx]
        sample      = all_chunks[i]
        book_num    = sample['book_num']
        para_num    = sample['para_num']

        system_prompt = SYSTEM_PROMPT_TEMPLATE.format(book_number=book_num, chapter_summary=SUMMARIES[book_num-1])
        messages = [
            {'role': 'system', 'content': system_prompt}
        ]
        for chunk, output_chunk in examples:
            input_text = render_chunk_message(chunk)
            output_text = output_chunk['atoms_raw']
            messages.append({'role': 'user', 'content': input_text})
            messages.append({'role': 'assistant', 'content': output_text})
        messages.append({'role': 'user', 'content': render_chunk_message(sample)})
        atoms_raw = gen_from_messages(messages)

        output = {
            'book_num': book_num,
            'para_num': para_num,
            'chunk_index': sample.get('chunk_index', None),
            'atoms_raw': atoms_raw
        }
        # print('Len0: ', len(chunks_processed))
        chunks_processed.append((sample, output))
        print('Processed chunk number: ', i, book_num, para_num)
        # print('Len1: ', len(chunks_processed))

        to_store.append(output)
        if len(to_store) > 5:
            append_data_jsonl(to_store, file_path)
            to_store = []

#### Extract structured atoms data (as json) from unstructured atoms text data

In [ ]:
import instructor
from pydantic import BaseModel
import openai
import pandas as pd
import anthropic
from instructor import Mode

def load_jsonl_to_dataframe(file_path, as_dataframe=False):
    data = []  # List to hold dictionaries
    try:
        with open(file_path, 'r') as file:
            for line in file:
                json_data = json.loads(line.strip())
                data.append(json_data)
    except FileNotFoundError:
        print(f"No file found at {file_path}")

    if as_dataframe:
        return pd.DataFrame(data) if data else pd.DataFrame()
    else:
        return data
    

# Define our desired output structure
class Atom(BaseModel):
    subject: str
    topic: str
    entities: list[str]
    text: str

class SpecializedTerminology(BaseModel):
    term: str
    definition: str

class Atoms(BaseModel):
    atoms: list[Atom]
    specialized_terminologies: list[SpecializedTerminology]

## Run

In [ ]:
from google.colab import drive

rendered_chunks = extract_chunks()

# Sort the rendered chunks based on book_num, para_num, and chunk_index (if exists)
sorted_chunks = sorted(rendered_chunks, key=lambda chunk: (
    chunk['book_num'],
    chunk['para_num'],
    chunk.get('chunk_index', 0)  # Use 0 as default value if chunk_index doesn't exist
))

drive.mount('/content/drive')
directory_path = '/content/drive/My Drive/AI_Data/aurelius/'
file_path = os.path.join(directory_path, 'chunks_processed.jsonl')
atoms_file_path = os.path.join(directory_path, 'atoms_processed.jsonl')

extract_atoms_as_raw_text(sorted_chunks, file_path, SEED)

data = load_jsonl_to_dataframe(file_path)

last_successful_idx = 0
atoms_data = []
for i in range(last_successful_idx, len(data)):
    x = data[i]
    raw = x['atoms_raw']
    atoms = instructor_client.chat.completions.create(
        model='gpt-3.5-turbo',
        messages=[
            {"role": "user", "content": raw},
        ],
        max_tokens=2048,
        response_model=Atoms,
    )

    atoms_data.append(atoms)
    append_data_jsonl([atoms.dict()], atoms_file_path)
    print('Processed: ', i)